In [1]:
raw_data_dir = '../data/raw/'
interim_data_dir = '../data/interim/'

In [2]:
import requests
import json
import pandas as pd

url= "https://crashviewer.nhtsa.dot.gov/CrashAPI"
#/crashes/GetCrashesByLocation?fromCaseYear=2014&toCaseYear=2015&state=1&county=1&format=json

fromCaseYear = "2010"
toCaseYear = "2020"
state = "6"
qurl = f"{url}/crashes/GetCrashesByLocation?fromCaseYear={fromCaseYear}&toCaseYear={toCaseYear}&state={state}&county=73&format=json"

cali = requests.get(qurl).json()

In [3]:
cali_df = pd.DataFrame(cali['Results'][0]).groupby('ST_CASE', as_index=False).max()

In [4]:
year = 2015
st_case = 60022

qurl = f"{url}/crashes/GetCaseDetails?stateCase={st_case}&caseYear={year}&state=6&format=json"
data = requests.get(qurl).json()
case = data['Results'][0][0]['CrashResultSet']


In [5]:
case.keys()

dict_keys(['ARR_HOUR', 'ARR_HOURNAME', 'ARR_MIN', 'ARR_MINNAME', 'CEvents', 'CF1', 'CF1NAME', 'CF2', 'CF2NAME', 'CF3', 'CF3NAME', 'CITY', 'CITYNAME', 'COUNTY', 'COUNTYNAME', 'CaseYear', 'DAY', 'DAY_WEEK', 'DAY_WEEKNAME', 'DRUNK_DR', 'FATALS', 'FUNC_SYS', 'FUNC_SYSNAME', 'HARM_EV', 'HARM_EVNAME', 'HOSP_HR', 'HOSP_HRNAME', 'HOSP_MN', 'HOSP_MNNAME', 'HOUR', 'HOURNAME', 'LATITUDE', 'LATITUDENAME', 'LGT_COND', 'LGT_CONDNAME', 'LONGITUD', 'LONGITUDNAME', 'MAN_COLL', 'MAN_COLLNAME', 'MILEPT', 'MILEPTNAME', 'MINUTE', 'MINUTENAME', 'MONTH', 'MonthName', 'NHS', 'NHSNAME', 'NOT_HOUR', 'NOT_HOURNAME', 'NOT_MIN', 'NOT_MINNAME', 'NPersons', 'NmCrashes', 'NmImpairs', 'NmPriors', 'PEDS', 'PERMVIT', 'PERNOTMVIT', 'PERSONS', 'PVH_INVL', 'ParkWorks', 'PbTypes', 'RAIL', 'RAILNAME', 'RD_OWNER', 'RD_OWNERNAME', 'RELJCT1', 'RELJCT1NAME', 'RELJCT2', 'RELJCT2NAME', 'REL_ROAD', 'REL_ROADNAME', 'ROAD_FNC', 'ROAD_FNCNAME', 'ROUTE', 'ROUTENAME', 'RUR_URB', 'RUR_URBNAME', 'SCH_BUS', 'SCH_BUSNAME', 'SP_JUR', 'SP_JUR

In [6]:
case['LONGITUD']

'-117.063127780'

In [7]:
def extract_people(v):
    for p in v['Persons']:
        yield {
            'Speed Limit Exceeded': v['SPEEDRELNAME'],
            'Speed limit': v['TRAV_SP'],
            'Vin Number': v['VINNAME'],
            'Traveled Speed Veh': v['VSPD_LIM'],
            'Make': v['MAKENAME'],
            'Make/Model': v['MAK_MODNAME'],
            'Model': v['MODELNAME'],
            'Type of Vehicle': v['BODY_TYPNAME'],
            "Driver ZIP Code": v['DR_ZIP'],
            **person_dict(p),
        }

def person_dict(p):
    return {
        "Age": p['AGE'],
        "Age Name": p['AGENAME'],
        "County": p['COUNTYNAME'],
        "Death Day of Month": p['DEATH_DANAME'],
        "DOA Name": p['DOANAME'],
        # injury sev
        "Injury Severity Name": p['INJ_SEVNAME'],
        "Race": p['RACENAME'],
        "Road Type": p["ROAD_FNCNAME"],
        "Sex": p["SEXNAME"],
        "Make": p["MAKENAME"],
    }

        
def get_people(case):

    hour = case['HOUR']
    minute = case['MINUTE']
    time = f"{hour}:{minute}"
        
    accident_info = {
        'Lng': case['LONGITUD'],
        'Lat': case['LATITUDE'],
        'Case Number': case['ST_CASE'],
        "Description of Veh Coll": case['CF2NAME'], 
        "Day of Week": case['DAY_WEEKNAME'],
        "Drunk Driver": case['DRUNK_DR'],
        "Year": case['CaseYear'],
        "Month": case['MonthName'],
        "Hour": hour,
        "Time of Accident": time,
    }

    vehicles = case['Vehicles']
    pk = 'NPersons'
    pedestrians = [] if (pk not in case.keys() or case[pk] is None) else case[pk] 
    
    people = [{**accident_info, **p} for v in vehicles for p in extract_people(v)] \
        + [{**accident_info, **person_dict(p)} for p in pedestrians]
    return pd.DataFrame(people)

def get_events(case):
    c_events = [{
        'Case Number': case['ST_CASE'],
# In a traffic accident AOI is Area of Impact. The spot the two cars collided is measured 
# to a fixed object, usually the curb, so it can be reconstructed later.
        'Area of Impact': e['AOI1NAME'],
# standard of evidence
# https://safety.fhwa.dot.gov/rsdp/cdip_rpti.aspx
        'Standard of Evenidence': e['SOENAME'],
        'Event Number': e['EVENTNUM'],
        'Vehicle 1': e['VNUMBER1'],
        'Vehicle 2': e['VNUMBER2'],
    } for e in case['CEvents']]
    
    return pd.DataFrame(c_events)




In [8]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from api_config import census_api_key
c = Census(census_api_key, year=2013)


class CensusData(object):
    
    census_cache = {}

    @classmethod
    def census_by_year(cls, year):

        if year in cls.census_cache:
            return cls.census_cache[year]

        file_path = f'{ interim_data_dir }census_{ year }'
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            if 'Year' not in df.columns:
                df['Year'] = year
                df.to_csv(file_path)
            return df

        try:
            census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                                  "B19301_001E",
                                  "B17001_002E"), {'for': 'zip code tabulation area:*'}, year=year)
        # Convert to DataFrame
            census_pd = pd.DataFrame(census_data)

        # Column Reordering
            census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                                  "B01002_001E": "Median Age",
                                                  "B19013_001E": "Household Income",
                                                  "B19301_001E": "Per Capita Income",
                                                  "B17001_002E": "Poverty Count",
                                                  "NAME": "Name",
                                                  "zip code tabulation area": "Zipcode"})
            census_pd['Year'] = str(year)
            
            census_pd.to_csv(file_path)
            return census_pd

        except:
            print('no data')
            return None
        
    @classmethod
    def all_years(cls):
        # load all census tables 2011 - 2018
        years = range(2011, 2019)
        all_years = [cls.census_by_year(y) for y in years]
        df = pd.concat(all_years, ignore_index=True)
        df['Zipcode'] = df['Zipcode'].astype('str')
        df['Year'] = df['Year'].astype('str')
        return df
    
    @classmethod
    def ensure_census_columns(cls, df):
        if 'Per Capita Income' in df.columns: return df
        merged = pd.merge(df, cls.all_years(), how='left', left_on=['Year', 'Accident ZIP'], right_on=['Year', 'Zipcode'])
        return merged


In [9]:
CensusData.all_years().dtypes

Unnamed: 0             int64
Unnamed: 0.1           int64
Median Age           float64
Population           float64
Poverty Count        float64
Household Income     float64
Per Capita Income    float64
Name                  object
Zipcode               object
Year                  object
dtype: object

In [10]:
import json
from shapely.geometry import shape, Point
# depending on your version, use: from shapely.geometry import shape, Point


class ZipCoder(object):
    
    def __init__(self):
        self.js = None
    
    def __get_zip(self, lat, lng):
        point = Point(lng, lat)

        for feature in self.js['features']:
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                zip_code = feature['properties']['zip']
                return zip_code


    def __row_to_zip(self, r):
        lat = float(r['Lat'])
        lng = float(r['Lng'])
        return self.__get_zip(lat, lng)

    
    def ensure_acc_zips(self, df):
        with open(f'{ raw_data_dir }Zip Codes.geojson') as f:
            self.js = json.load(f)
            
        acc_zip_col = 'Accident ZIP'

        if acc_zip_col not in df.columns: 
            zip_codes = df.apply(self.__row_to_zip, axis=1)
            df[acc_zip_col] = zip_codes
            

In [11]:
from IPython.display import clear_output
import grequests
from itertools import islice
import os


def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())

# LARGE CHUNK SIZE WILL BLOW UP SERVER AND CAUSING: AttributeError: 'NoneType' object has no attribute 'json'
chunk_size = 5
fromCaseYear = "2010"
toCaseYear = "2020"
state = "6"
case_file_base = raw_data_dir

data_lists = {}


def url_from_row(r):
    statecase = r["ST_CASE"]
    caseyear = r["CaseYear"]
    return f"{url}/crashes/GetCaseDetails?stateCase={statecase}&caseYear={caseyear}&state=6&format=json"


def get_file_path(case):
    return f'{ case_file_base }{ case["ST_CASE"] }.json'
    
    
def load_case(file_path):
    with open(file_path, 'r') as f:
        case = json.load(f)
        return case

    
def __get_cases():
    urls = []
    found_locally = 0
    for i, r in cali_df.iterrows():
        file_path = get_file_path(r)
        if os.path.exists(file_path):
            found_locally += 1
            clear_output(wait=True)
            print(f'{ found_locally } files found locally')
            yield load_case(file_path)
        else:
            url = url_from_row(r)
            urls.append(url)
    print(f'{ len(urls) } need to be fetched. ')
    for c in __chunk_and_fetch(urls):
        yield c
    

def __fetch_cases(urls):
    rs = (grequests.get(u) for u in urls)
    case_data = grequests.map(rs)
    return [data.json()['Results'][0][0]['CrashResultSet'] for data in case_data]
    
    
def __save_case(case):
    file_path = get_file_path(case)
    with open(file_path, 'w') as json_file:
        json.dump(case, json_file)
    
    
def __chunk_and_fetch(urls):
    chunked = chunk(urls, chunk_size)
    
    i = 0
    for chunked_urls in chunked:
        i += 1
        clear_output(wait=True)
        print(f'Retrieving chunk { i } of { len(urls) / chunk_size } ...')
        cases = __fetch_cases(chunked_urls) 
        for case in cases:
            __save_case(case)
            yield case

        
people_key = 'people'
events_key = 'events'


def __get_case_lists():
    # actualize list to avoid redundant api calls
    case_list = list(__get_cases())
    
    file_path_people = f"{ interim_data_dir }people.csv"
    people_list = [get_people(case) for case in case_list]
    people_df = pd.concat(people_list, ignore_index=True, sort=False)
    people_df.to_csv(file_path_people)
    data_lists[people_key] = people_df
    
    file_path_events = f"{ interim_data_dir }events.csv"
    event_list = [get_events(case) for case in case_list]
    event_df = pd.concat(event_list, sort=False)
    event_df.to_csv(file_path_events)
    data_lists[events_key] = event_df

    return people_df, event_df
    

def __ensure_updates(df):
    ZipCoder().ensure_acc_zips(df)
    merged = CensusData.ensure_census_columns(df)
    
    unnecessary_columns = [
        'Unnamed: 0', 'Unnamed: 0.1',
        'Unnamed: 0_x', 'Unnamed: 0_x', 
        'Unnamed: 0.1_x', 'Unnamed: 0.1.1', 
        'Unnamed: 0_y', 'Unnamed: 0.1_y']
    for c in unnecessary_columns:
        if c in df.columns:
            merged.drop(c,
                1,
                inplace=True)
            
    renames = {
        'ZIP Code': 'Driver ZIP Code',
    }
    
    for k in renames.keys():
        if k in merged.columns:
            merged.rename(columns=renames, inplace=True)
    
    file_path = f"{ interim_data_dir }people.csv"
    merged.to_csv(file_path)
    
    return merged
    
    
def get_people_list():
    cached = get_cached_list(people_key)
    if cached is not None:
        return __ensure_updates(cached)
    df = __get_case_lists()[0]
    return __ensure_updates(df)


def get_event_list():
    cached = get_cached_list(events_key)
    if cached is not None:
        return cached
    df = __get_case_lists()[1]
    file_path = f"{ interim_data_dir }events.csv"
    df.to_csv(file_path)
    return df


def get_cached_list(key):
    if key in data_lists:
        return data_lists[key]
    
    file_path = f"{ interim_data_dir }{ key }.csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        data_lists[key] = 
        \df
        return df
    
    return None

C:\Users\Mike\Anaconda3\lib\site-packages\grequests.py:21: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.contrib.pyopenssl (C:\\Users\\Mike\\Anaconda3\\lib\\site-packages\\urllib3\\contrib\\pyopenssl.py)', 'urllib3.util (C:\\Users\\Mike\\Anaconda3\\lib\\site-packages\\urllib3\\util\\__init__.py)']. 
  curious_george.patch_all(thread=False, select=False)
C:\Users\Mike\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [12]:
df = get_people_list()
grouped = df.groupby('Case Number').count()
assert len(cali_df) == len(grouped)

In [12]:
# Import downloaded data from https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/.
# Use dtype="object" to match other
#zip_latlng = pd.read_csv("zip_latlng.csv", dtype="object")
zip_latlng = pd.read_csv("../Data/interim/zip_latlng.csv")

zip_latlng = zip_latlng.rename(columns={"Zip": "Zipcode"})

# Visualize
zip_latlng.head()

,Zipcode,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint,Unnamed: 8
0,71937,Cove,AR,34.398483,-94.39398,-6,1,34.398483,-94.39398
1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,35.624351,-92.16056
2,56171,Sherburn,MN,43.660847,-94.74357,-6,1,43.660847,-94.74357
3,49430,Lamont,MI,43.010337,-85.89754,-5,1,43.010337,-85.89754
4,52585,Richland,IA,41.194129,-91.98027,-6,1,41.194129,-91.98027


In [13]:
get_people_list()['Accident ZIP']

0       92108.0
1       92108.0
2       92028.0
3       92154.0
4       92154.0
5       92019.0
6       92019.0
7       92019.0
8       92107.0
9       92107.0
10      92107.0
11      92107.0
12      92104.0
13      92029.0
14      91941.0
15      91950.0
16      91950.0
17      92065.0
18      92065.0
19      92065.0
20      92071.0
21      92071.0
22      91911.0
23      91911.0
24      91911.0
25      91911.0
26      91911.0
27      91911.0
28      91911.0
29      91911.0
         ...   
3996    91941.0
3997    91941.0
3998    91941.0
3999    91941.0
4000    91941.0
4001    91941.0
4002    91941.0
4003    92020.0
4004    92021.0
4005    92021.0
4006    92021.0
4007    92021.0
4008    92021.0
4009    92021.0
4010    92126.0
4011    91911.0
4012    91911.0
4013    91911.0
4014    91911.0
4015    92084.0
4016    91911.0
4017    91911.0
4018    91911.0
4019    91911.0
4020    91911.0
4021    92025.0
4022    92025.0
4023    91945.0
4024    91910.0
4025    92101.0
Name: Accident ZIP, Leng

In [14]:
# Merge the two data sets along zip code
data_complete = pd.merge(
    zip_latlng, census_pd, how="left", on=["Zipcode", "Zipcode"])

# Remove rows missing data
data_complete = data_complete.dropna()
print(len(data_complete))

31675


In [15]:
sd_cty_zip = [
92152,
92196,
92192,
92161,
92132,
92193,
92143,
92138,
92182,
92198,
92150,
92199,
92038,
92140,
92112,
92093,
92145,
92092,
92091,
92014,
92173,
92027,
92118,
92119,
92124,
92106,
92107,
92116,
92139,
92029,
92071,
92113,
92102,
92104,
92025,
92037,
92120,
92110,
91945,
92129,
92105,
92103,
92131,
92114,
92117,
91942,
92128,
92111,
92122,
91932,
92109,
92126,
92127,
92123,
92108,
92115,
92121,
92154,
92130,
92101,
91911]

data_complete_dropneg = data_complete.loc[(data_complete["Per Capita Income"] > 0) &
                                          (data_complete["Zipcode"].isin(sd_cty_zip)),
                                          :]

# Visualize
print(len(data_complete_dropneg))
data_complete_dropneg.head()

#data_complete.to_csv("data_complete.csv")

45


,Zipcode,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint,Unnamed: 8,Unnamed: 0,Median Age,Population,Poverty Count,Household Income,Per Capita Income,Name
415,92114,San Diego,CA,32.706954,-117.05420,-8,1,32.706954,-117.05420,28505.0,33.2,69798.0,11762.0,56679.0,18915.0,ZCTA5 92114
3115,92108,San Diego,CA,32.773600,-117.13785,-8,1,32.773600,-117.13785,28500.0,32.0,20548.0,3080.0,65398.0,40431.0,ZCTA5 92108
3881,92139,San Diego,CA,32.677286,-117.05241,-8,1,32.677286,-117.05241,28525.0,33.1,36934.0,4956.0,58050.0,20371.0,ZCTA5 92139
4169,92102,San Diego,CA,32.714992,-117.12537,-8,1,32.714992,-117.12537,28494.0,30.7,45461.0,13290.0,41840.0,19543.0,ZCTA5 92102
6499,92118,Coronado,CA,32.682727,-117.17441,-8,1,32.682727,-117.17441,28509.0,39.8,22939.0,1207.0,90256.0,50693.0,ZCTA5 92118


In [16]:
import gmaps

# Import API key
from api_config import g_key

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

ImportError: cannot import name 'g_key' from 'api_config' (C:\Users\Mike\Google Drive\ucsd_extension\assignments\ucsd_bootcamp-traffic_accident_analysis\notebooks\api_config.py)

In [17]:
# Store 'Lat' and 'Lng' into  locations 
locations = data_complete_dropneg[["Latitude", "Longitude"]].astype(float)

# Convert income and age to float and store
# HINT: be sure to handle NaN values
income = data_complete_dropneg["Per Capita Income"].astype(float)
#age = data_complete_dropneg["Median Age"].astype(float)
#data_complete_dropneg["Per Capita Income"].value_counts()

In [ ]:
# Create an income Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=income, 
                                 dissipating=True,
                                 point_radius = 10)

fig.add_layer(heat_layer)

fig

In [ ]:
income_symbol_layer = gmaps.symbol_layer(
    locations, fill_color='blue',
    stroke_color='blue', scale=3,
#    locations_accident, fill_color='rgba(0, 150, 0, 0.4)',
#    stroke_color='rgba(0, 0, 150, 0.4)', scale=3,
#    info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
)

fig = gmaps.figure()
fig.add_layer(income_symbol_layer)

fig

In [ ]:
people_df_new = pd.read_csv("people.csv")
people_df_new.columns

In [ ]:
people_df_new['Case Count'] = ""

In [ ]:
people_df_new2 = people_df_new.groupby(['Case Number'], as_index=False).agg({'Case Count': 'count', 'Lat': 'first', 'Lng': 'first'})
people_df_new2

In [ ]:
# Convert accident frequency to list
top_accidents = people_df_new2.nlargest(10, "Case Count")
top_accidents

In [ ]:
locations_accident = top_accidents[["Lat", "Lng"]].astype(float)

accident_rate = top_accidents['Case Count'].tolist()
#name = hotel_df['Hotel Name'].tolist()
#city = hotel_df['City'].tolist()
#country = hotel_df['Country'].tolist()

#fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure()

# Assign the marker layer to a variable
#markers = gmaps.marker_layer(locations_accident, info_box_content=hotel_info)
markers = gmaps.marker_layer(locations_accident)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

In [ ]:
# Create accident layer
accident_layer = gmaps.symbol_layer(
    locations_accident, fill_color='red',
    stroke_color='red', scale=2,
#    locations_accident, fill_color='rgba(0, 150, 0, 0.4)',
#    stroke_color='rgba(0, 0, 150, 0.4)', scale=3,
#    info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
)


fig = gmaps.figure()
fig.add_layer(accident_layer)

fig

In [ ]:
# Create an accident Heatmap layer
fig = gmaps.figure()

accident_heat_layer = gmaps.heatmap_layer(locations_accident, weights=accident_rate, 
                                          dissipating=True,
                                          point_radius = 10, 
                                         )


fig.add_layer(accident_heat_layer)

fig

In [ ]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)            # weighted by income
#fig.add_layer(markers)              # top number of accidents
fig.add_layer(accident_layer)        # top number of accidents
#fig.add_layer(income_symbol_layer)

fig